In [1]:
# Import Statements
import pyrealsense2 as rs
import numpy as np
import cv2
from pyzbar import pyzbar
import math
import copy
import re

In [2]:
# Newton Raphson Functions
def DCreator(Guess, LenA, A, B, C, t):
    c = 299792.458 # km/s
    D = np.zeros((LenA, 4))
    for i in range(LenA):
        D[i] = [2*(Guess[0][0]-A[i]), 2*(Guess[1][0]-B[i]), 2*(Guess[2][0]-C[i]), 2*((c**2)*(t[i]-Guess[3][0]))]
    return D

def FCreator(Guess, LenA, A, B, C, t):
    c = 299792.458
    F = np.zeros((LenA, 1))
    for i in range(LenA):
        F[i] = (Guess[0][0]-A[i])**2 + (Guess[1][0]-B[i])**2 + (Guess[2][0]-C[i])**2 - (c*(t[i]-Guess[3][0]))**2
    return F

# Guess: 1 x #QR
# A: 1 x #QR
# B: 1 x #QR
# C: 1 x #QR
# t: 1 x #QR
# LA: float
def MVNewton(Guess, A, B, C, t, LA):
    GuessTrans = np.array([[Guess[0]],[Guess[1]],[Guess[2]],[Guess[3]]])
    c = 299792.458
    
    F = FCreator(GuessTrans, LA, A, B, C, t) 
    D = DCreator(GuessTrans, LA, A, B, C, t)
    
    for i in range(20):
        V = np.linalg.lstsq(D, F*-1)
        GuessTrans = GuessTrans + V[0]
        F = FCreator(GuessTrans, LA, A, B, C, t)
        D = DCreator(GuessTrans, LA, A, B, C, t)
    
    return GuessTrans

# G_ = [0,0,0,0] # initial guess
# A_ = [1,2,3] # x-coordinates
# B_ = [1,2,3] # y-coordinates
# C_ = [0,0,0] # z-coordinates
# T_ = [1,2,3] # time-offset

# G_ = [0,0,670,0] # initial guess
# A_ = [15600,18760,17610,19170] # x-coordinates
# B_ = [7540,2750,14630,610] # y-coordinates
# C_ = [20140,18610,13480,18390] # z-coordinates
# T_ = [0.07074,0.07220,0.07690,0.07242] # time-offset

# print(MVNewton(G_, A_, B_, C_, T_, len(A_)))

In [3]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30) # depth stream
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30) # color stream
align = rs.align(rs.stream.color) # align both streams to same pov

In [4]:
# Enable visualizer and filters for later use
colorizer = rs.colorizer()
spatial = rs.spatial_filter()
spatial.set_option(rs.option.filter_magnitude, 5)
spatial.set_option(rs.option.filter_smooth_alpha, 1)
spatial.set_option(rs.option.filter_smooth_delta, 50)
spatial.set_option(rs.option.holes_fill, 3)
hole_filling = rs.hole_filling_filter()
depth_to_disparity = rs.disparity_transform(True)
disparity_to_depth = rs.disparity_transform(False)

In [ ]:
# Start streaming
profile = pipeline.start(config)

try:
    while True:

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        frames = align.process(frames)
        
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # filter depth stream: depth2disparity -> spatial -> disparity2depth -> hole_filling
        depth_frame = depth_to_disparity.process(depth_frame)
        depth_frame = spatial.process(depth_frame)
        depth_frame = disparity_to_depth.process(depth_frame)
        depth_frame = hole_filling.process(depth_frame)

        # Convert images to numpy arrays
        depth_image = np.asanyarray(colorizer.colorize(depth_frame).get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        # detect and decode barcodes
        object_depth = np.asanyarray(depth_frame.get_data())
        barcode_poses = []
        barcodes = pyzbar.decode(color_image)
        
        X_ = []
        Y_ = []
        Z_ = []
        D_ = []
        T_ = []
        
        for barcode in barcodes:
            (x, y, w, h) = barcode.rect
            cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.rectangle(depth_image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            barcodeData = barcode.data.decode("utf-8")
            barcodeType = barcode.type
            
            curr = copy.deepcopy(object_depth)
            curr = curr[math.floor(((x)+(x+w))/2-1):math.ceil(((x)+(x+w))/2+1),math.floor(((y)+(y+h))/2-1):math.ceil(((y)+(y+h))/2+1)].astype(float)
            depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
            curr = curr * depth_scale
            dist,_,_,_ = cv2.mean(curr)
            barcode_poses.append((barcodeData, dist))
 
            # draw the barcode data and barcode type on the image
            text = "{} ({})".format(barcodeData, dist)
            cv2.putText(color_image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            # store poses of codes
            curr_pose = re.split(',|\(|\)',barcodeData)
            X_.append(float(curr_pose[1]))
            Y_.append(float(curr_pose[2]))
            Z_.append(float(curr_pose[3]))
            D_.append(dist)
            T_.append((dist/(10**3))/299792.458)
        
        if barcodes:
            curr_cam_pos = MVNewton([0,0,0,0], X_, Y_, Z_, T_, len(barcodes))
            
            text_pos_x = 'X: ' + str(curr_cam_pos[0][0])
            text_pos_y = 'Y: ' + str(curr_cam_pos[1][0])
            text_pos_z = 'Z: ' + str(curr_cam_pos[2][0])
            text_pos_error = 'Error: ' + str(curr_cam_pos[3][0])
            
            cv2.putText(depth_image, text_pos_x, (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(depth_image, text_pos_y, (20, 420), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(depth_image, text_pos_z, (20, 440), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(depth_image, text_pos_error, (20, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
           
        # Stack both images horizontally
        images = np.hstack((color_image, depth_image))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        key = cv2.waitKey(1)
        
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
        
finally:

    # Stop streaming
    pipeline.stop()

c:\users\neelo\anaconda3\envs\opencv-env\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
